# Convolutional Neural Network

Convolution
<br>
fn: (f * g)(t) = integral from -inf to inf f(τ)g(t-τ)dτ
<br>
Input image + feature detector/kernel/filter (usuallu 3x3) -> convolution -> feature map by multiplying each small matrix (overlapping) in input image with filter (element * element) by moving through step (stride) -> convolved feature -> reduced size of image (high stride = smaller size)
<br>
Use diff filters -> multiple feature maps -> have lots of versions of the image -> model decides which to use where

ReLU
applied on convolution layer -> increase non-linearity (images are non linear, necessary to break up linearity introduced by our process) -> turns all negatives to 0

Pooling/DownSampling
<br>
diff positions and orientations -> recognise feature -> spatial invariance -> identify feature even if its distorted
<br>
take a small part of feature map, take max value and put in pooled featur map -> move box by stride(1/2 pixels) -> feature preserved (max number = where input image and feature detector matched more), removing almost 75% of unwanted info, accounting for distortion by considering max value, reducing number of params, prevent overfitting
<br>
max pooling (taking max), mean/sum/sub-sampling

Flattening
<br>
pooled feature map -> flatten to column (row by row) -> input layer for ann

Full Connection
<br>
Adding ann to cnn, fully conn hidden layers, combine features into more attributes to predict the classes, output per class (no. of outputs not 1)
<br>
input image -> convolved, pooled, flattened -> go through ann -> prediction -> wrong pred -> error calc (loss function) -> back prop -> adjustments to optimise -> weights and feature detectors adjusted
<br>
Output neurons -> 2 classes = 2 outputs -> ans neuron gets activated, other ignores -> identify neurons that contribute to identifying features of that particular output -> give pred for each class

Softmax
<br>
Final prob add upto 1 -> since output neurons don't know the value of each other -> softmax fn -> out put of 2 neurons = z1 and z2 -> apply softmax fn (fj(z) = e^zj/sum of all k e^zk) -> normalised exponential fn -> squashes a k-dim vector of arbitrary values to a k-dim vector of real values bw 0 and 1 that add up to 1
<br>
<br>
Cross-Entropy
<br>
Li = -log(e^fyi/sum of all j e^fj)
<br>
H(p,q) = - sum of all x p(x)logq(x)
<br>
prob goes to q, 1/0 goes to p
<br>
Classification error = number of wrong pred/total number of pred
<br>
MSE = average of sum of squared errors
<br>
Cross Entropy = formula, helps assessing small errors (due to presence of log fn) since at the beginning, back-prop would not yield much due to small values

### Importing the libraries

In [2]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
# Image Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Feature scaling to each pixel
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255
)
test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3]))

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

## Part 4 - Making a single prediction

In [ ]:
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size=(64,64))
# convert to array
test_image = image.img_to_array(test_image)
# create batches
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image)
# 1=dog, 0=cat
training_set.class_indices
if result[0][0] == 1:
  pred = 'dog'
else:
  pred = 'cat'
print(pred)